<a href="https://colab.research.google.com/github/yiruchen1993/nvidia_gtc_dli_rapids_2020/blob/section_notebooks%2Fdata_manipulation/1_06_prep_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備資料以建立圖表

為達到本實作坊更大的資料科學目標需求，我們將使用能反映整個英國道路網的資料。在開始時，我們會使用一份擷取自官方 [GML](https://en.wikipedia.org/wiki/Geography_Markup_Language) 檔案，以表格式 csv 格式呈現的道路資料。最終我們將使用 cuGraph 針對此資料執行 GPU 加速圖表分析，然而若要這麼做，我們需要預先處理資料，才能建立圖表。

在此學習筆記中，你將學習其他 cuDF 資料轉換技巧，例如在示範如何準備初始資料供 cuGraph 擷取時，我們會教你如何使用合併功能。接著，你會進行一連串練習，以類似方式轉換資料，建立邊緣權重不同的圖表。

## 目標

完成此學習筆記後，你將能夠:

- 使用 cuDF 執行 GPU 加速合併作業
- 準備資料供 cuGraph 擷取
- 建立 GPU 加速圖表

## 匯入

除了 `cudf` 外，在此學習筆記中，我們也將匯入 `cugraph`，以便我們在準備資料後建立 GPU 加速圖表。我們也將匯入 `networkx`，以便稍後進行簡要效能比較。

In [ ]:
import cudf
import cugraph as cg

import networkx as nx

## 讀取資料

在此學習筆記中，我們將使用兩筆資料來源，幫助我們建立英國道路網圖表。

### 英國道路節點

第一筆資料表格會說明道路網中的節點: 端點、交點 (含圓環)，以及分散表示一整條蜿蜒長路的各點，以便正確繪製圖表 (以免繪製出直線)。

每點的坐標皆使用我們稍早在 1-05 節中學過的 OSGB36 格式。

In [ ]:
road_nodes = cudf.read_csv('./data/road_nodes_1-06.csv')

In [ ]:
road_nodes.dtypes

node_id     object
east       float64
north      float64
type        object
dtype: object

In [ ]:
road_nodes.shape

(3121148, 4)

In [ ]:
road_nodes.head()

,node_id,east,north,type
0,id02FE73D4-E88D-4119-8DC2-6E80DE6F6594,320608.0938,870994.0000,junction
1,id634D65C1-C38B-4868-9080-2E1E47F0935C,320628.5000,871103.8125,road end
2,idDC14D4D1-774E-487D-8EDE-60B129E5482C,320635.4688,870983.9375,junction
3,id51555819-1A39-4B41-B0C9-C6D2086D9921,320648.7188,871083.5625,junction
4,id9E362428-79D7-4EE3-B015-0CE3F6A78A69,320658.1875,871162.3750,junction


In [ ]:
print(road_nodes['type'].unique())

0       junction
1    pseudo node
2       road end
3     roundabout
Name: type, dtype: object


### 英國道路邊緣

第二個資料表格則說明道路區段，包含其開始與結束點、長度以及類型。

In [ ]:
road_edges = cudf.read_csv('./data/road_edges_1-06.csv')

In [ ]:
road_edges.dtypes

src_id    object
dst_id    object
length     int64
type      object
form      object
dtype: object

In [ ]:
road_edges.shape

(3725531, 5)

In [ ]:
road_edges.head()

,src_id,dst_id,length,type,form
0,#id138447A5-91D4-4642-BFAC-13F309705429,#id84C9DAD4-9243-4742-B582-E8CBC848E08A,314,Restricted Local Access Road,Single Carriageway
1,#idD615F9C5-5BE9-412D-9FED-F4928BAB4146,#idA1BB20B9-0751-4B42-9925-20607ABF5027,104,Restricted Local Access Road,Single Carriageway
2,#idDC14D4D1-774E-487D-8EDE-60B129E5482C,#id51555819-1A39-4B41-B0C9-C6D2086D9921,100,Restricted Local Access Road,Single Carriageway
3,#id626FC567-199C-41FB-9F29-1AB718874128,#idACD1B0A9-F870-4B46-88CF-C870A9EDAF8B,93,Restricted Local Access Road,Single Carriageway
4,#id03312900-B147-4CA3-A858-E2BF6AD1ECA7,#id02FE73D4-E88D-4119-8DC2-6E80DE6F6594,95,Restricted Local Access Road,Single Carriageway


In [ ]:
print(road_edges['type'].unique())

0                          A Road
1                          B Road
2               Local Access Road
3                      Local Road
4                      Minor Road
5                        Motorway
6    Restricted Local Access Road
7           Secondary Access Road
Name: type, dtype: object


In [ ]:
print(road_edges['form'].unique())

0    Collapsed Dual Carriageway
1              Dual Carriageway
2                 Guided Busway
3                    Roundabout
4        Shared Use Carriageway
5            Single Carriageway
6                     Slip Road
Name: form, dtype: object


## 練習: 建立相容 ID

我們的 csv 檔源自原始 [GML](https://en.wikipedia.org/wiki/Geography_Markup_Language) 檔案，且如上所示，`road_edges['src_id']` 和 `road_edges['dst_id']` 的開頭皆包含 `#` 字元，但`road_nodes['node_id']` 卻未包含此字元。若要使邊緣與節點的 ID 相容，請使用 cuDF 的 [字串方法](https://docs.rapids.ai/api/nvstrings/stable/) `.str.lstrip` 取代 `road_edges` 中的 `src_id` 與 `dst_id` 欄，去除欄值的開頭字元 `#`。

### 解決方案

In [ ]:
# %load solutions/make_ids_compatible
road_edges['src_id'] = road_edges['src_id'].str.lstrip('#')
road_edges['dst_id'] = road_edges['dst_id'].str.lstrip('#')
road_edges[['src_id', 'dst_id']].head()


,src_id,dst_id
0,id138447A5-91D4-4642-BFAC-13F309705429,id84C9DAD4-9243-4742-B582-E8CBC848E08A
1,idD615F9C5-5BE9-412D-9FED-F4928BAB4146,idA1BB20B9-0751-4B42-9925-20607ABF5027
2,idDC14D4D1-774E-487D-8EDE-60B129E5482C,id51555819-1A39-4B41-B0C9-C6D2086D9921
3,id626FC567-199C-41FB-9F29-1AB718874128,idACD1B0A9-F870-4B46-88CF-C870A9EDAF8B
4,id03312900-B147-4CA3-A858-E2BF6AD1ECA7,id02FE73D4-E88D-4119-8DC2-6E80DE6F6594


## 練習: 移除重複值

我們的資料原先來自多個地圖圖格，且橫跨圖格邊界的道路在其開始與結束圖格上皆已記錄。

為 `road_nodes` 與 `road_edges` 使用 `drop_duplicates` 方法，以移除重複值。

#### 解決方案

In [ ]:
# %load solutions/remove_duplicates
road_nodes = road_nodes.drop_duplicates().reset_index(drop=True)
road_edges = road_edges.drop_duplicates().reset_index(drop=True)


## 資料摘要

清理資料後，我們可以看看目前要處理多少條道路和端點/交點/曲線點，及其占用的 GPU 記憶體容量。

In [ ]:
print(f'{road_edges.shape[0]} edges, {road_nodes.shape[0]} nodes')

3667200 edges, 3078117 nodes


In [ ]:
!nvidia-smi

Wed Oct  7 05:31:57 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 000091FC:00:00.0 Off |                    0 |
| N/A   31C    P0    39W / 250W |   2313MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  Off  | 0000A964:00:00.0 Off |                    0 |
| N/A   30C    P0    25W / 250W |     12MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

## 準備資料以建立道路網圖表

我們稍後將使用 cuGraph 圖表建立工具預估邊緣來源、邊緣目的地以及邊緣權重 (非必要)。邊緣來源與目的地必須為從 0 到 *N*-1 的 32 位元整數精簡間隔，而 *N* 代表圖表中的節點數量。

此外，在 cuGraph 中，無指向性的邊緣應儲存為具兩種方向的指向性邊緣。由於我們的資料並未指出單行道，我們將假設所有的道路皆可視為雙向道 (無指向)。

我們需要些許準備才能應對這些限制，在稍後探索時就能以超高速度進行分析。

目前，與我們的需求恰好相反，`road_edges` 資料架構包含 `src_id` 與 `dst_id` 字串，可與 `road_nodes` 中的 `node_id` 值對應。此外，即便是雙向道路，我們的道路邊緣也會記錄為從一個來源前往單一目的地。

In [ ]:
road_edges[['src_id', 'dst_id']].dtypes

src_id    object
dst_id    object
dtype: object

In [ ]:
road_nodes[['node_id']].dtypes

node_id    object
dtype: object

In [ ]:
road_edges['src_id'][0]

'id000000F5-5180-4C03-B05D-B01352C54F89'

In [ ]:
road_edges['dst_id'][0]

'id0ABF4CB2-6997-4952-A9AA-313DE2EF6538'

In [ ]:
road_nodes['node_id'][0]

'id000000F5-5180-4C03-B05D-B01352C54F89'

為了準備資料來使用 cuGraph 建立圖表，我們必須完成下列步驟:

1.為每個 `node_id` 建立唯一的 int32 值，我們稱之為該節點的 `graph_id`，且會將對應結果從 `node_id` 儲存為 `graph_id`。每當我們分析圖表獲得結果時，就可以使用對應功能來尋找哪個原始節點對應至 `graph_id`。
2.使用對應功能，將每個邊緣的開始點與結束點的 `road_edges` 更新為 `graph_id`。我們最終將呼叫來源點 `graph_src` 與目的地點 `graph_dst`。
3.建立具備相同邊緣的全新資料架構，但互換來源與目的地點，再串連互換的資料架構與原始值和刪除重複值 (以便在圖表中將每條道路當成雙向道)。

### 建立圖表 ID

如上所述，我們的首要步驟是建立名為 `node_id_to_graph_id_mapping` 的全新資料架構，其中包含所有現有 `node_id` 值，以及具備唯一 int32 值的 `graph_id` 欄。

In [ ]:
node_id_to_graph_id_mapping = cudf.DataFrame()

unique_node_ids = road_edges['src_id'].append(road_edges['dst_id']).unique()
node_id_to_graph_id_mapping['node_id'] = unique_node_ids

現在，則是建立新的 `graph_id` 欄，其中每個 `node_id` 都需包含唯一 int32 值的精簡間隔，而我們可以單純使用預設的 `existing_node_ids.index` 建立欄。它們已經是 int64 值，我們唯一需要做的就是轉換成 int32:

In [ ]:
node_id_to_graph_id_mapping['graph_id'] = cudf.Series(unique_node_ids.index).astype('int32')

In [ ]:
node_id_to_graph_id_mapping.head()

,node_id,graph_id
0,id000000F5-5180-4C03-B05D-B01352C54F89,0
1,id000003F8-9E09-4829-AD87-6DA4438D22D8,1
2,id000010DA-C89A-4198-847A-6E62815E038A,2
3,id000017A0-1843-4BC7-BCF7-C943B6780839,3
4,id00001B2A-155F-4CD3-8E06-7677ADC6AF74,4


In [ ]:
node_id_to_graph_id_mapping.dtypes

node_id     object
graph_id     int32
dtype: object

### 將 `graph_id` 合併至 `road_nodes`

cuDF 與 Pandas 一樣，皆提供合併功能。即便我們未直接需要使用 `road_nodes` 建立圖表，但稍後在實作坊中也需要此資訊，以便分析道路的地理空間位置。我們現在會透過合併將 `graph_id` 欄新增至 `road_nodes`:

In [ ]:
%time road_nodes = road_nodes.merge(node_id_to_graph_id_mapping, on='node_id', how='left')

CPU times: user 90 ms, sys: 118 ms, total: 208 ms
Wall time: 213 ms


In [ ]:
road_nodes.head()

,node_id,east,north,type,graph_id
0,id040D1AA6-DECA-4E49-8B79-077932272CBB,450737.0000,506461.000,road end,48736
1,id040D1BFB-B7A3-45DB-B7B3-B547CC135AEA,506833.0000,102916.000,junction,48737
2,id040D26F4-D78C-4077-9DA6-5FC560C6A5D6,433340.0000,416171.000,road end,48738
3,id040D3244-20A1-447C-98BC-AED626CCF1BE,413415.3750,429897.625,junction,48739
4,id040D3D22-832F-49EF-A8A8-6F53188CC87F,241705.2969,599384.375,junction,48740


#### 為 `road_nodes` 重新建立索引

為了方便稍後有效率地查詢，我們會為 `road_nodes` 重新建立索引，並以`graph_id` 作為索引。切記，就 `graph_id` 而言，我們通常會從圖表分析取得結果，這個步驟能讓我們輕鬆獲得節點的其他資訊。我們接著要在此新索引上分類資料架構:

In [ ]:
road_nodes = road_nodes.set_index(road_nodes.graph_id)
%time road_nodes = road_nodes.sort_index()

CPU times: user 218 ms, sys: 85.5 ms, total: 303 ms
Wall time: 307 ms


In [ ]:
road_nodes.head()

,node_id,east,north,type,graph_id
graph_id,,,,,
0,id000000F5-5180-4C03-B05D-B01352C54F89,432920.250,572547.4375,road end,0
1,id000003F8-9E09-4829-AD87-6DA4438D22D8,526616.375,189678.3906,junction,1
2,id000010DA-C89A-4198-847A-6E62815E038A,336879.000,731824.0000,junction,2
3,id000017A0-1843-4BC7-BCF7-C943B6780839,380635.000,390153.0000,junction,3
4,id00001B2A-155F-4CD3-8E06-7677ADC6AF74,337481.000,350509.7188,junction,4


### 將 `graph_id` 合併至 `road_edges`

我們現在需要將 `graph_id` 合併至 `road_edges` 兩次: 一次是為了建立新的 int32 來源 ID 欄；另一次是建立新的 int32 目的地 ID 欄。

#### `src_id` 的新 int32 欄

In [ ]:
node_id_to_graph_id_mapping.head()

,node_id,graph_id
0,id000000F5-5180-4C03-B05D-B01352C54F89,0
1,id000003F8-9E09-4829-AD87-6DA4438D22D8,1
2,id000010DA-C89A-4198-847A-6E62815E038A,2
3,id000017A0-1843-4BC7-BCF7-C943B6780839,3
4,id00001B2A-155F-4CD3-8E06-7677ADC6AF74,4


In [ ]:
road_edges.head()

,src_id,dst_id,length,type,form
0,id000000F5-5180-4C03-B05D-B01352C54F89,id0ABF4CB2-6997-4952-A9AA-313DE2EF6538,44,Local Road,Single Carriageway
1,id000003F8-9E09-4829-AD87-6DA4438D22D8,id8B9308E8-DD5C-4C5B-AE92-A8F6C471B551,70,Secondary Access Road,Single Carriageway
2,id000003F8-9E09-4829-AD87-6DA4438D22D8,idCE926C8D-A698-410F-BE88-0EDE56BB0476,40,Local Road,Single Carriageway
3,id000010DA-C89A-4198-847A-6E62815E038A,id000010DA-C89A-4198-847A-6E62815E038A,55,Local Road,Single Carriageway
4,id000017A0-1843-4BC7-BCF7-C943B6780839,idECFC5CB9-4FD9-4377-8D68-00B1D2654547,71,Local Road,Single Carriageway


In [ ]:
# Add `graph_id` value into `road_edges` from `node_id_to_graph_id_mapping` row with `src_id` `node_id` match.
%time road_edges = road_edges.merge(node_id_to_graph_id_mapping, how='left', left_on='src_id', right_on='node_id')

CPU times: user 240 ms, sys: 226 ms, total: 466 ms
Wall time: 505 ms


如你所見，`road_edges` 現在包含我們想要的 `graph_id` 欄:

In [ ]:
road_edges[['graph_id']].dtypes

graph_id    int32
dtype: object

In [ ]:
road_edges['graph_id'].head()

0    2173
1    2173
2    2174
3    2175
4    2176
Name: graph_id, dtype: int32

#### `dst_id` 的新 int32 欄

我們現在將為 `dst_id` 重複上方步驟:

In [ ]:
# Add second `graph_id` value into `road_edges` from `node_id_to_graph_id_mapping` row with `dst_id` `node_id` match.
%time road_edges = road_edges.merge(node_id_to_graph_id_mapping, how='left', left_on='dst_id', right_on='node_id')

CPU times: user 318 ms, sys: 277 ms, total: 595 ms
Wall time: 620 ms


road_edges### 重新命名新欄

In [ ]:
road_edges.head()

,src_id,dst_id,length,type,form,node_id_x,graph_id_x,node_id_y,graph_id_y
0,id00EA30C9-A088-4453-A201-17ABFE0CD4AA,id0107CE0C-A616-434B-94E5-8156223CE1B3,265,B Road,Single Carriageway,id00EA30C9-A088-4453-A201-17ABFE0CD4AA,11119,id0107CE0C-A616-434B-94E5-8156223CE1B3,12454
1,id00EA3496-2BF2-43DD-9AE5-297EE4F970C7,id1E59F9E9-9398-4116-96F4-05815579A23D,68,Secondary Access Road,Single Carriageway,id00EA3496-2BF2-43DD-9AE5-297EE4F970C7,11120,id1E59F9E9-9398-4116-96F4-05815579A23D,364511
2,id00EA34DA-92F6-4AE0-AD0D-0F37240591B8,idA1C3C8C8-AAB1-4ADC-AF96-CCB275D53D64,162,Local Road,Single Carriageway,id00EA34DA-92F6-4AE0-AD0D-0F37240591B8,11121,idA1C3C8C8-AAB1-4ADC-AF96-CCB275D53D64,1944954
3,id00EA364A-4D2B-40B9-A978-00DF24CB0C7A,id5C0DF1F3-6B44-4114-B3FE-8219CF06DB48,62,Restricted Local Access Road,Single Carriageway,id00EA364A-4D2B-40B9-A978-00DF24CB0C7A,11122,id5C0DF1F3-6B44-4114-B3FE-8219CF06DB48,1106148
4,id00EA51F2-B616-43F5-9FCD-0CFF6A02D20A,idB6EA8C58-AFCF-4C20-AD27-E5686417AFAC,397,Local Road,Single Carriageway,id00EA51F2-B616-43F5-9FCD-0CFF6A02D20A,11125,idB6EA8C58-AFCF-4C20-AD27-E5686417AFAC,2198570


第二次合併 `graph_id` 時，cuDF 會自動重新命名欄以避免重複，因此我們現在會同時擁有 `graph_id_x` 與 `graph_id_y` 欄:

In [ ]:
road_edges.dtypes

src_id        object
dst_id        object
length         int64
type          object
form          object
node_id_x     object
graph_id_x     int32
node_id_y     object
graph_id_y     int32
dtype: object

讓我們將其重新命名為更有意義的 `graph_src` 與 `graph_dst`:

In [ ]:
road_edges.rename({'graph_id_x': 'graph_src', 'graph_id_y': 'graph_dst'}, inplace=True)

In [ ]:
road_edges.dtypes

src_id       object
dst_id       object
length        int64
type         object
form         object
node_id_x    object
graph_src     int32
node_id_y    object
graph_dst     int32
dtype: object

In [ ]:
road_edges[['src_id', 'graph_src', 'dst_id', 'graph_dst']].head()

,src_id,graph_src,dst_id,graph_dst
0,id00EA30C9-A088-4453-A201-17ABFE0CD4AA,11119,id0107CE0C-A616-434B-94E5-8156223CE1B3,12454
1,id00EA3496-2BF2-43DD-9AE5-297EE4F970C7,11120,id1E59F9E9-9398-4116-96F4-05815579A23D,364511
2,id00EA34DA-92F6-4AE0-AD0D-0F37240591B8,11121,idA1C3C8C8-AAB1-4ADC-AF96-CCB275D53D64,1944954
3,id00EA364A-4D2B-40B9-A978-00DF24CB0C7A,11122,id5C0DF1F3-6B44-4114-B3FE-8219CF06DB48,1106148
4,id00EA51F2-B616-43F5-9FCD-0CFF6A02D20A,11125,idB6EA8C58-AFCF-4C20-AD27-E5686417AFAC,2198570


### 使用建立圖表所需的欄建立資料架構

在建立無指向性的圖表資料前，讓我們建立只包含 cuGraph 建立工具所需欄的較小資料架構，免於反覆移動不必要的資料:

In [ ]:
graph_prep = cudf.DataFrame()
graph_prep['src'] = road_edges['graph_src']
graph_prep['dst'] = road_edges['graph_dst']
graph_prep['length'] = road_edges['length'].astype('float') # cuGraph expects edge weights as floating points
print(graph_prep.shape)
print(graph_prep.dtypes)

(3667200, 3)
src         int32
dst         int32
length    float64
dtype: object


### 建立無指向性的圖表

我們現在要建立互換來源與目的地的新資料架構，以製作無指向性的圖表，並將新資料架構串連至 `graph_prep`，然後排除任何重複值。

In [ ]:
rev_gdf = cudf.DataFrame()

rev_gdf['src'] = graph_prep['dst']
rev_gdf['dst'] = graph_prep['src']
rev_gdf['length'] = graph_prep['length']

# Concatenate `graph_prep` and `rev_gdf`
graph_prep = cudf.concat([graph_prep, rev_gdf], 
                              ignore_index=True)
graph_prep.shape

(7334400, 3)

雖然我們可以讓道路邊緣擁有相同的開始與結束節點，但長度不同 (例如，一個從主幹線中斷或重新聯結到主幹線的環狀道路)，為了簡化，我們將刪除邊緣清單中的重複值，同時忽略 `length`，以便讓每對相連節點都只有一組 (無指向性) 邊緣。

In [ ]:
graph_prep.drop_duplicates(subset=['src', 'dst'], inplace=True)
graph_prep.shape

(7218169, 3)

讓我們確認 `graph_prep` 包含 int32 ID 的精簡間隔:

In [ ]:
# The min src or dst ID is 0
graph_prep[['src', 'dst']].min().min() == 0

True

In [ ]:
# The max src or dst id is the number of nodes - 1
graph_prep[['src', 'dst']].max().max() == unique_node_ids.count() - 1

True

In [ ]:
# The ID dtypes are int32s
graph_prep[['src', 'dst']].dtypes

src    int32
dst    int32
dtype: object

In [ ]:
graph_prep

,src,dst,length
28256,0,129165,44.0
28257,1,1678323,70.0
6487960,1,2372610,18.0
28258,1,2483057,40.0
28259,2,2,55.0
...,...,...,...
3649501,3078114,3057280,672.0
6909265,3078115,2721612,94.0
5708688,3078116,1721226,76.0
3649502,3078116,2979684,138.0


## 使用 cuGraph 建立圖表

我們現在已正確準備 `graph_prep`，就可以使用 cuGraph 建立圖表，以便進行加速分析了。若要做到這點，我們首先須要使用 cuGraph 具現化 `Graph` 執行個體，接著傳遞執行個體來源、邊緣目的地、邊緣權重 (這在我們的資料中是道路長度):

In [ ]:
G = cg.Graph()
%time G.add_edge_list(graph_prep['src'], graph_prep['dst'], graph_prep['length'])

/opt/conda/envs/rapids/lib/python3.6/site-packages/cugraph/structure/graph.py:191: UserWarning: add_edge_list will be deprecated in next release. Use from_cudf_edgelist instead
  Use from_cudf_edgelist instead')


CPU times: user 602 ms, sys: 106 ms, total: 708 ms
Wall time: 708 ms


你可以看到整個準備資料與建立圖表的過程有多迅速，就算我們從頭開始，重新啟動核心，按一下此文字，然後從 [Run] (執行) 選單中選取 [Run All Above Selected Cell] (執行上述所選的所有儲存格) 也一樣 (應該花不到十秒)。
在 Network X 中，我們也使用經過清理與準備的相同 Pandas 資料架構建立相同圖表，以供比較。這應該需要花費大約一分鐘，你可以在程序處理時繼續向下閱讀。

In [ ]:
graph_prep_cpu = graph_prep.to_pandas()

%time G_cpu = nx.convert_matrix.from_pandas_edgelist(graph_prep_cpu, source='src', target='dst', edge_attr='length')

CPU times: user 43.8 s, sys: 2.81 s, total: 46.6 s
Wall time: 46.6 s


### 分析圖表

既然圖表已建立完成，我們現在就可以分析其中的節點與邊緣數量了:

In [ ]:
G.number_of_nodes()

3078117

In [ ]:
G.number_of_edges()

7218169

我們還可以分析圖表節點的角度:

In [ ]:
deg_df = G.degree()

我們複製了每個邊緣，以便建立無指向性的圖表，預期節點至少有 2 度。

In [ ]:
deg_df['degree'].describe()[1:]

mean     4.689990
std      1.913452
min      2.000000
25%      2.000000
50%      6.000000
75%      6.000000
max     16.000000
Name: degree, dtype: float64

稍後在實作坊中，你將花更多時間使用此 GPU 加速圖表。

## 練習: 建立包含時間權重的道路圖表

針對這一連串練習，你將使用我們剛剛介紹的技巧建立並分析英國道路的新圖表，但這一次，你要運用的不是邊緣權重的原始距離，而是 2 節點間的交通時間。

你將從我們先前使用的 `road_edges` 資料架構開始此練習，此資料架構中已包含我們在 `graph_src` 與 `graph_dst` 建立圖表所需的唯一 int32 `graph_id` 值。

In [ ]:
road_edges.dtypes

src_id       object
dst_id       object
length        int64
type         object
form         object
node_id_x    object
graph_src     int32
node_id_y    object
graph_dst     int32
dtype: object

### 透過道路類型轉換速度

為了計算通過道路的交通時間，我們必須先知道速限。我們將按照各類型道路的速限規則使用 `road_edges['type']`，以計算交通時間。

以下是我們資料中的唯一道路類型:

In [ ]:
road_edges['type'].unique()

0                          A Road
1                          B Road
2               Local Access Road
3                      Local Road
4                      Minor Road
5                        Motorway
6    Restricted Local Access Road
7           Secondary Access Road
Name: type, dtype: object

而這是我們用來轉換速度所需的速限假設表格:

In [ ]:
# https://www.rac.co.uk/drive/advice/legal/speed-limits/
# Technically, speed limits depend on whether a road is in a built-up area and the form of carriageway,
# but we can use road type as a proxy for built-up areas.
# Values are in mph.

speed_limits = {'Motorway': 70,
               'A Road': 60,
               'B Road': 60,
               'Local Road': 30,
               'Local Access Road': 30,
               'Minor Road': 30,
               'Restricted Local Access Road': 30,
               'Secondary Access Road': 30}

我們首先要建立 `speed_gdf`，以儲存每種道路類型的速限。

In [ ]:
speed_gdf = cudf.DataFrame()

speed_gdf['type'] = speed_limits.keys()
speed_gdf['limit_mph'] = [speed_limits[key] for key in speed_limits.keys()]
speed_gdf

,type,limit_mph
0,Motorway,70
1,A Road,60
2,B Road,60
3,Local Road,30
4,Local Access Road,30
5,Minor Road,30
6,Restricted Local Access Road,30
7,Secondary Access Road,30


接著，我們將新增 `limit_m/s` 欄，以便我們為每種道路類型評估通過速度 (單位為公尺/秒)。

In [ ]:
# We will have road distances in meters (m), so to get road distances in seconds (s), we need to multiply by meters/mile and divide by seconds/hour
# 1 mile ~ 1609.34 m
speed_gdf['limit_m/s'] = speed_gdf['limit_mph'] * 1609.34 / 3600
speed_gdf

,type,limit_mph,limit_m/s
0,Motorway,70,31.292722
1,A Road,60,26.822333
2,B Road,60,26.822333
3,Local Road,30,13.411167
4,Local Access Road,30,13.411167
5,Minor Road,30,13.411167
6,Restricted Local Access Road,30,13.411167
7,Secondary Access Road,30,13.411167


### 步驟 1: 將 `speed_gdf` 合併至 `road_edges`

因為我們需要使用 `road_edges` 中的值來建立圖表，我們需要將 `speed_gdf` 合併至 `road_edges`。你可以聯結這兩種資料架構都通用的 `type` 欄。

In [ ]:
# road_edges_new = road_edges.merge(speed_gdf, how='left', on='type')

In [ ]:
# road_edges_new.head()

,src_id,dst_id,length,type,form,node_id_x,graph_src,node_id_y,graph_dst,limit_mph,limit_m/s
0,id003B7E37-119B-44F4-BCF2-51DB8451E70B,id16527908-002A-41F5-844A-53AAD2B66F28,44,Local Road,Single Carriageway,id003B7E37-119B-44F4-BCF2-51DB8451E70B,2809,id16527908-002A-41F5-844A-53AAD2B66F28,268182,30,13.411167
1,id003B7F17-BF47-4C1D-BF65-A825D0C38EE6,idC7ABE34D-275E-41DE-9403-059EBD6C215C,120,Minor Road,Single Carriageway,id003B7F17-BF47-4C1D-BF65-A825D0C38EE6,2810,idC7ABE34D-275E-41DE-9403-059EBD6C215C,2400343,30,13.411167
2,id003B7F17-BF47-4C1D-BF65-A825D0C38EE6,idF78A47FA-435F-42AC-8B8E-8DC03F0DAAC7,46,Restricted Local Access Road,Single Carriageway,id003B7F17-BF47-4C1D-BF65-A825D0C38EE6,2810,idF78A47FA-435F-42AC-8B8E-8DC03F0DAAC7,2976038,30,13.411167
3,id003B8310-CCD0-4161-8736-A73996DB64ED,id46174298-1637-4313-AC7F-5153D650BB85,79,Local Road,Single Carriageway,id003B8310-CCD0-4161-8736-A73996DB64ED,2811,id46174298-1637-4313-AC7F-5153D650BB85,842098,30,13.411167
4,id003B8310-CCD0-4161-8736-A73996DB64ED,id601ABE78-00C6-435B-AFDF-C9F12DAD8966,97,Local Road,Single Carriageway,id003B8310-CCD0-4161-8736-A73996DB64ED,2811,id601ABE78-00C6-435B-AFDF-C9F12DAD8966,1155301,30,13.411167


#### 解決方案

In [ ]:
# %load solutions/merge_speed_gdf
road_edges = road_edges.merge(speed_gdf, on='type')
road_edges.dtypes


src_id        object
dst_id        object
length         int64
type          object
form          object
node_id_x     object
graph_src      int32
node_id_y     object
graph_dst      int32
limit_mph      int64
limit_m/s    float64
dtype: object

### 步驟 2: 在「秒」欄中新增長度

現在，你需要計算通過指定道路所需的實際時間。你可以將道路長度除以速限 (m/s) 以求得答案。請在 `road_edges` 上執行此計算，並將結果儲存至新欄 `length_s`。

In [ ]:
# length_s = road_edges_new.length/road_edges_new['limit_m/s']

In [ ]:
# road_edges_new['length_s'] = length_s

In [ ]:
# road_edges_new.head()

,src_id,dst_id,length,type,form,node_id_x,graph_src,node_id_y,graph_dst,limit_mph,limit_m/s,length_s
0,id003B7E37-119B-44F4-BCF2-51DB8451E70B,id16527908-002A-41F5-844A-53AAD2B66F28,44,Local Road,Single Carriageway,id003B7E37-119B-44F4-BCF2-51DB8451E70B,2809,id16527908-002A-41F5-844A-53AAD2B66F28,268182,30,13.411167,3.280848
1,id003B7F17-BF47-4C1D-BF65-A825D0C38EE6,idC7ABE34D-275E-41DE-9403-059EBD6C215C,120,Minor Road,Single Carriageway,id003B7F17-BF47-4C1D-BF65-A825D0C38EE6,2810,idC7ABE34D-275E-41DE-9403-059EBD6C215C,2400343,30,13.411167,8.947767
2,id003B7F17-BF47-4C1D-BF65-A825D0C38EE6,idF78A47FA-435F-42AC-8B8E-8DC03F0DAAC7,46,Restricted Local Access Road,Single Carriageway,id003B7F17-BF47-4C1D-BF65-A825D0C38EE6,2810,idF78A47FA-435F-42AC-8B8E-8DC03F0DAAC7,2976038,30,13.411167,3.429978
3,id003B8310-CCD0-4161-8736-A73996DB64ED,id46174298-1637-4313-AC7F-5153D650BB85,79,Local Road,Single Carriageway,id003B8310-CCD0-4161-8736-A73996DB64ED,2811,id46174298-1637-4313-AC7F-5153D650BB85,842098,30,13.411167,5.890614
4,id003B8310-CCD0-4161-8736-A73996DB64ED,id601ABE78-00C6-435B-AFDF-C9F12DAD8966,97,Local Road,Single Carriageway,id003B8310-CCD0-4161-8736-A73996DB64ED,2811,id601ABE78-00C6-435B-AFDF-C9F12DAD8966,1155301,30,13.411167,7.232779


#### 解決方案

In [ ]:
# %load solutions/length_in_seconds
road_edges['length_s'] = road_edges['length'] / road_edges['limit_m/s']
road_edges['length_s'].head()


0     5.741484
1     7.680167
2    60.695689
3    12.154051
4    10.439062
Name: length_s, dtype: float64

### 步驟 3: 建立準備資料架構

請製作名為 `exercise_graph` 的新資料架構，並將其新增至三個欄: `src`、`dst` 與 `length_s`，你可以從 `road_edges` 中獲得上述所有資訊。

In [ ]:
# exercise_graph = cudf.DataFrame()

# exercise_graph['src'] = road_edges_new['src_id']
# exercise_graph['dst'] = road_edges_new['dst_id']
# exercise_graph['length_s'] = road_edges_new['length_s']

# exercise_graph.head()

,src,dst,length_s
0,id003B7E37-119B-44F4-BCF2-51DB8451E70B,id16527908-002A-41F5-844A-53AAD2B66F28,3.280848
1,id003B7F17-BF47-4C1D-BF65-A825D0C38EE6,idC7ABE34D-275E-41DE-9403-059EBD6C215C,8.947767
2,id003B7F17-BF47-4C1D-BF65-A825D0C38EE6,idF78A47FA-435F-42AC-8B8E-8DC03F0DAAC7,3.429978
3,id003B8310-CCD0-4161-8736-A73996DB64ED,id46174298-1637-4313-AC7F-5153D650BB85,5.890614
4,id003B8310-CCD0-4161-8736-A73996DB64ED,id601ABE78-00C6-435B-AFDF-C9F12DAD8966,7.232779


In [ ]:
exercise_graph.dtypes

src          object
dst          object
length_s    float64
dtype: object

#### 解決方案

In [ ]:
# %load solutions/prep_dataframe
exercise_graph = cudf.DataFrame()

exercise_graph['src'] = road_edges['graph_src']
exercise_graph['dst'] = road_edges['graph_dst']
exercise_graph['length_s'] = road_edges['length_s']

print(exercise_graph.shape)
exercise_graph.dtypes


(3667200, 3)


src           int32
dst           int32
length_s    float64
dtype: object

### 步驟 4: 建立無指向性的邊緣

建立反轉 `src` 與 `dst` 的資料架構，使`exercise_graph` 無指向性，並將其與 `exercise_graph` 串連。請務必刪除重複其他列 `src` 與 `dst` 值的列。

In [ ]:
# rev_gdf = cudf.DataFrame()

# rev_gdf['src'] = exercise_graph['dst']
# rev_gdf['dst'] = exercise_graph['src']
# rev_gdf['length_s'] = exercise_graph['length_s']

# exercise_graph = cudf.concat([exercise_graph, rev_gdf], 
#                               ignore_index=True)

In [ ]:
# exercise_graph.drop_duplicates(subset=['src', 'dst'], inplace=True)
# print(exercise_graph.shape)

# # The maximum graph_id is the number of nodes - 1
# print(exercise_graph[['src', 'dst']].max().max() == road_nodes['node_id'].unique().shape[0] - 1)

# # The minimum graph_id is 0 
# print(exercise_graph[['src', 'dst']].min().min() == 0)

# # The ID dtypes are int32s
# print(exercise_graph[['src', 'dst']].dtypes == 'int32')

(1, 3)
False
True
src    False
dst    False
dtype: bool


#### 解決方案

In [ ]:
# %load solutions/make_undirected
rev_gdf = cudf.DataFrame()

rev_gdf['src'] = exercise_graph['dst']
rev_gdf['dst'] = exercise_graph['src']
rev_gdf['length_s'] = exercise_graph['length_s']

exercise_graph = cudf.concat([exercise_graph, rev_gdf], 
                              ignore_index=True)

print(exercise_graph.shape)

exercise_graph.drop_duplicates(subset=['src', 'dst'], inplace=True)
print(exercise_graph.shape)

# The maximum graph_id is the number of nodes - 1
print(exercise_graph[['src', 'dst']].max().max() == road_nodes['node_id'].unique().shape[0] - 1)

# The minimum graph_id is 0 
print(exercise_graph[['src', 'dst']].min().min() == 0)

# The ID dtypes are int32s
print(exercise_graph[['src', 'dst']].dtypes == 'int32')


(7334400, 3)
(7218169, 3)
True
True
src    True
dst    True
dtype: bool


### 步驟 5: 建立圖表

使用在 `exercise_graph` 中找到的來源與目的地，以及在邊緣權重「秒」值中找到的長度，來建立名為 `G_ex` 的 cuGraph 圖表。

#### 解決方案

In [ ]:
# %load solutions/construct_graph
G_ex = cg.Graph()
G_ex.add_edge_list(exercise_graph['src'], exercise_graph['dst'], exercise_graph['length_s'])


/opt/conda/envs/rapids/lib/python3.6/site-packages/cugraph/structure/graph.py:191: UserWarning: add_edge_list will be deprecated in next release. Use from_cudf_edgelist instead
  Use from_cudf_edgelist instead')


## 下一步

在下一份學習筆記中，你將使用等同於英國人口數 5 倍的資料集作業，此資料集的大小超過單一 GPU 的記憶體容量。為了處理這筆資料，你將使用 Dask cuDF，將資料分割到 4 個 GPU 上，並執行你在較小單一 GPU 資料集上透過 Vanilla cuDF 進行的相同類型資料操作。

<br>
<div align="center"><h2>請重新啟動核心</h2></div>